In [10]:
#import relevant libraries

import requests
import json
from pprint import pprint
import operator
import numpy
import pandas 

# from datetime module
from datetime import date
from datetime import timedelta
from pathlib import Path  

# My Team info
team_id = 3402291


/var/folders/sz/1kc6hj850ps8d9bbrtzsswcw0000gn/T/ipykernel_22802/1194701558.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


### Overall Player Transfer Data

In [16]:
# Request Information on all players today (now?)
base_url = 'https://fantasy.premierleague.com/api/'
r_all_players_today = requests.get(base_url+'bootstrap-static/').json()
pprint(r_all_players_today, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10609194}


In [17]:
# Total Active players in the game at this point in time (estimate)

boolean_list_of_all_events_status = list(map(operator.itemgetter('is_current'), r_all_players_today['events']))
current_event = [item for item, condition in zip(r_all_players_today['events'], boolean_list_of_all_events_status) if condition]
current_gameweek = current_event[0].get('id')
total_active_players_estimate = int(round(r_all_players_today['total_players']*(0.97)**current_gameweek,))

In [18]:
# Net transfers of players at this moment 

all_players_transfers_in = list(map(operator.itemgetter('transfers_in'), r_all_players_today['elements']))
all_players_transfers_out = list(map(operator.itemgetter('transfers_out'), r_all_players_today['elements']))
net_transfers_overall_today = numpy.array(all_players_transfers_in) - numpy.array(all_players_transfers_out)

In [19]:
# Price Change so far this event 

all_players_price_change_for_current_event = list(map(operator.itemgetter('cost_change_event'), r_all_players_today['elements']))

In [20]:
# Players prices today

all_players_prices_today = list(map(operator.itemgetter('now_cost'), r_all_players_today['elements']))

In [21]:
# Player currently flagged red

list_of_all_players_status = list(map(operator.itemgetter('status'), r_all_players_today['elements']))
boolean_list_of_players_injured = [True if x == 'i' else False for x in list_of_all_players_status]

In [48]:
# Create DataFrame from dictionary of all this transfer data

price_change_dict = {
    "price_change_this_night": 0, #tomorrow, we define this as today_data.player_prices_today - yesterday_data.player_prices_today
    "net_transfers_in_out_since_last_price_change": 0, #tomorrow, we define this as yesterday_data.net_transfers_in_out_since_last_price_change + today_data_net_transfers_in_out_since_yesterday, unless price change occurs, then we reset to ="Net Transfers In/Out since yesterday
    "net_transfers_in_out_since_yesterday": 0, #tomorrow, we define this as today_data.net_transfers_in_out_overall_as_of_today - yesterday_data.net_transfers_in_out_overall_as_of_today
    "price_change_so_far_for_this_event ": all_players_price_change_for_current_event,
    "total_active_players_estimate": total_active_players_estimate,
    "players_injured": boolean_list_of_players_injured,
    "player_prices_today" : all_players_prices_today,
    "net_transfers_in_out_overall_as_of_today" : net_transfers_overall_today, #not to be used in modelling
    "player_ids" : numpy.array(range(1,len(boolean_list_of_players_injured)+1))
}

# Dictionary into DataFrame
today_data = pandas.DataFrame(price_change_dict)
today_data.set_index("player_ids", inplace = True)
today_data

# Player transfer data is updated at least every 40 minutes, and probably every 30 or even every 15 minutes
# This dataframe is a snapshot at 8pm GMT Jan 29th

,price_change_this_night,net_transfers_in_out_since_last_price_change,net_transfers_in_out_since_yesterday,price_change_so_far_for_this_event,total_active_players_estimate,players_injured,player_prices_today,net_transfers_in_out_overall_as_of_today
player_ids,,,,,,,,
1,0,0,0,0,5596143,False,44,-52470
2,0,0,0,0,5596143,False,39,-7584
3,0,0,0,0,5596143,False,44,-6296
4,0,0,0,0,5596143,False,54,-7147
5,0,0,0,0,5596143,False,50,-1054891
...,...,...,...,...,...,...,...,...
786,0,0,0,0,5596143,False,40,828
787,0,0,0,0,5596143,False,45,29
788,0,0,0,0,5596143,False,45,175


In [32]:
# Get today's date
today = date.today() #+ timedelta(days = 1) #REMEMBER TO ADD A DAY WHEN YOU DO ANALYSIS IF IT IS BEFORE 7PM!!!
 
# Get yesterday date
yesterday = today - timedelta(days = 1) #+ timedelta(days = 1)

Today is:  2024-01-29
Yesterday was:  2024-01-28


In [43]:
#update yesterday's dataframe with today's results 

#get yesterday's saved results 
yesterday_filepath = Path('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/Pipeline-Steps/Saved_DataFrames/{}/{}.csv'.format(yesterday,yesterday))
yesterday_data = pandas.read_csv(yesterday_filepath)

#updates 
yesterday_data.price_change_this_night = today_data.player_prices_today - yesterday_data.player_prices_today
yesterday_data.net_transfers_in_out_since_yesterday = today_data.net_transfers_in_out_overall_as_of_today - yesterday_data.net_transfers_in_out_overall_as_of_today


# Update net transfer since last price change in today's data (including reset rule if price change occured overnight)
today_data.net_transfers_in_out_since_last_price_change = yesterday_data.net_transfers_in_out_since_last_price_change + yesterday_data.net_transfers_in_out_since_yesterday
boolean_player_changes = [today_data.price_change_this_night != 0]
unchanged_tranfer_data = today_data.net_transfers_in_out_since_last_price_change
changed_array = [0 if b else a for a, b in zip(unchanged_tranfer_data, boolean_player_changes)]
today_data.net_transfers_in_out_since_last_price_change = changed_array

#rewrite yesterday's file 
yesterday_data.to_csv(yesterday_filepath) 

In [ ]:
#save today's dataframe to today's folder

today_filepath = Path('/Users/thomasribaroff/Documents/GitHub/FullPipeline-FPLPricePredictor/Pipeline-Steps/Saved_DataFrames/{}/{}.csv'.format(today,today))  
today_data.to_csv(filepath) 

### Personal Team Data

In [ ]:
# Request Information on my team

r_my_team = requests.get(base_url+'entry/{}/event/{}/picks/'.format(team_id,current_gameweek)).json()
my_players_ids = list(map(operator.itemgetter('element'), r_my_team['picks']))
my_players_ids

# Get list of all player IDs
total_list_of_players_ids = list(map(operator.itemgetter('id'), r_all_players_today['elements']))

# Makes a list of all players that is TRUE is player is in my team
boolean_list_of_players_in_my_team = [player in my_players_ids for player in total_list_of_players_ids]

# Use boolean list to exact all info about my players this week
my_players_total = [item for item, condition in zip(r_all_players_today['elements'], boolean_list_of_players_in_my_team) if condition]
my_players_total

# Print all my player's names
for players in my_players_total:
    print(players.get('web_name'))